# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [1]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "69o6i0JqwgpLM5a5v4NN"
client_secret = "dFeCQMyNQd"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
    #    response_body = response.read()
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())
    #    print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [2]:
#print(news_data)
print(len(news_data))
#print(news_data[0])
#print(news_data[0]['items'])
#print(len(news_data[0]['items']))
#print(news_data[0]['items'][2])
#print(news_data[0]['items'][7]['link'])

3


In [3]:
naver_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
        #print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)
        

# 사이트 확인하기에 편한 코드 구조.
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=009&aid=0004437628
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002171545
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003401677
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=031&aid=0000510621
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004479660
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=076&aid=0003474702


In [4]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [5]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('driver/chromedriver')

In [6]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=009&aid=0004437628
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002171545


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003401677
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=031&aid=0000510621


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004479660
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=076&aid=0003474702

['온·오프 교육 스타트업 러닝스푼즈 투자유치', '구글, 기계학습 플랫폼 ‘텐서플로 2.0’ 정식 출시']
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    유명 스타트업 투자자· CFO  등 50억 밸류   \u3000성인들을 대상으로 직무교육 서비스를 온·오프라인으로 제공하고 있는 러닝스푼즈 (대표 이창민)가 사모투자 방식으로 2억원 규모 투자를 유치했다고 밝혔다. \u3000이번 라운드는 프리시리즈A 단계로 유명 스타트업 투자자 및  CFO  등 다양한 전문가들이 참여했다. 투자후 포스트 밸류는 약 50억원이다. \u3000러닝스푼즈 모토는 \'성장이 필요한 순간, 러닝스푼즈 !\' 다. 러닝스푼즈의 교육 범위는 파이낸스, 비즈니스, 데이터 사이언스, 부동산, 원데이 클래스 등 다양하다. 특히 온라인 교육 뿐 아니라 오프라인을 통해 심층적인 교육 서비스를 제공하고 있다. 아울러 삼성증권, 한화증권,  LG 화학 등 대기업들을 대상으로 하는 사내 교육도 진행하고 있다. \u3000러닝스푼즈는 "올해 상반기 파이썬과 R을 활용한 데이터 분석 및 시각화, 파이썬을 활용한 금융데이터 수집 및 분석 등 디지털 트랜스포메이션( Digital   Transformation )에 기반한 융합형 과정을 국내 유수의 기업들에게 제공했다"고 설명했다. 지난 2년 동안 러닝스푼즈는 파이낸스 영역으로 시작해 다양한 카테고리들로 빠르게 확장하고 있다. \u3000러닝스푼즈는 이번 투자를 통해 채용과 연계한 전일제 교육 등 새로운 사업을 

In [7]:
print(naver_news_title[0])

['온·오프 교육 스타트업 러닝스푼즈 투자유치', '구글, 기계학습 플랫폼 ‘텐서플로 2.0’ 정식 출시']


In [8]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    유명 스타트업 투자자· CFO  등 50억 밸류   \u3000성인들을 대상으로 직무교육 서비스를 온·오프라인으로 제공하고 있는 러닝스푼즈 (대표 이창민)가 사모투자 방식으로 2억원 규모 투자를 유치했다고 밝혔다. \u3000이번 라운드는 프리시리즈A 단계로 유명 스타트업 투자자 및  CFO  등 다양한 전문가들이 참여했다. 투자후 포스트 밸류는 약 50억원이다. \u3000러닝스푼즈 모토는 \'성장이 필요한 순간, 러닝스푼즈 !\' 다. 러닝스푼즈의 교육 범위는 파이낸스, 비즈니스, 데이터 사이언스, 부동산, 원데이 클래스 등 다양하다. 특히 온라인 교육 뿐 아니라 오프라인을 통해 심층적인 교육 서비스를 제공하고 있다. 아울러 삼성증권, 한화증권,  LG 화학 등 대기업들을 대상으로 하는 사내 교육도 진행하고 있다. \u3000러닝스푼즈는 "올해 상반기 파이썬과 R을 활용한 데이터 분석 및 시각화, 파이썬을 활용한 금융데이터 수집 및 분석 등 디지털 트랜스포메이션( Digital   Transformation )에 기반한 융합형 과정을 국내 유수의 기업들에게 제공했다"고 설명했다. 지난 2년 동안 러닝스푼즈는 파이낸스 영역으로 시작해 다양한 카테고리들로 빠르게 확장하고 있다. \u3000러닝스푼즈는 이번 투자를 통해 채용과 연계한 전일제 교육 등 새로운 사업을 시도 할 예정이다. 특히 투자금은 올해 하반기에 열릴 스타트업 채용과 연계한 부트캠프 코스에 사용될 예정이다. 부트캠프 코스는 평일에는 파이썬과 R과 같은 데이터 분석을 위한 언어를 공부하고, 주말에는 데이터분석가 혹은 퍼포먼스 마케터 등 각자의 커리어패스에 맞는 전공심화 과정을 제공할 예정이다. \u3000이창민 러닝스푼즈 대표는 "교육의 퀄리티를 쓰는 동시에 성장을 추구하고 있다. 투자금을 마케팅 자금으로 소비하

In [9]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

2
2


In [10]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)